In [4]:
import pandas as pd
import numpy as np

#  Load the dataset
df = pd.read_csv("opensea_nft_sales.csv", low_memory=False)

#  Remove duplicate records
df.drop_duplicates(inplace=True)

#  Filter out rarely used payment tokens (less than 50 occurrences)
min_threshold = 50
token_counts = df['payment_token.name'].value_counts(dropna=False)
filtered_tokens = token_counts[token_counts >= min_threshold].index
df = df[df['payment_token.name'].isin(filtered_tokens)]

print(f"Remaining number of tokens: {len(filtered_tokens)}")
print(f"Filtered dataset shape: {df.shape}")

#  Fill missing asset names with contextual placeholders
df['asset.name'] = df.apply(
    lambda row: f"Unnamed in {row['asset.collection.name']}" if pd.isna(row['asset.name']) and pd.notna(row['asset.collection.name'])
    else ("Unnamed Asset" if pd.isna(row['asset.name']) else row['asset.name']),
    axis=1
)

#  Fill missing collection names based on asset name or default to "Unknown Collection"
df['asset.collection.name'] = df.apply(
    lambda row: f"Collection of {row['asset.name']}" if pd.isna(row['asset.collection.name']) and pd.notna(row['asset.name'])
    else ("Unknown Collection" if pd.isna(row['asset.collection.name']) else row['asset.collection.name']),
    axis=1
)

#  Drop irrelevant or unused columns
df.drop(columns=['asset.collection.short_description', 'asset.permalink'], inplace=True)

#  Fill missing payment token names with the most common one
most_common_token = df['payment_token.name'].mode()[0]
df['payment_token.name'] = df['payment_token.name'].fillna(most_common_token)

#  Standardize token names using a mapping dictionary
token_map = {
    'Ether': 'ETH',
    'Wrapped Ether': 'WETH',
    'USD Coin': 'USDC',
    'Dai Stablecoin': 'DAI',
    'Decentraland MANA': 'MANA',
    'Matic Token': 'MATIC',
    'Enjin Token': 'ENJ',
    'Gala': 'GALA'
    # Can be extended as needed
}
df['payment_token.name'] = df['payment_token.name'].replace(token_map)

#  Convert total_price from string to float and from Wei to ETH
df['total_price'] = pd.to_numeric(df['total_price'], errors='coerce') / 1e18

#  Load token price reference file
token_price_df = pd.read_csv("token_prices.csv")
token_price_df.columns = token_price_df.columns.str.strip()
price_dict = dict(zip(token_price_df['token_name'], token_price_df['usd_price']))

#  Keep only transactions with known token prices
df = df[df['payment_token.name'].isin(price_dict.keys())]

#  Calculate the USD equivalent of each transaction
df['payment_token.usd_price'] = df['payment_token.name'].map(price_dict)
df['usd_price'] = df['total_price'] * df['payment_token.usd_price']

#  Drop rows where USD price could not be calculated
df = df[df['usd_price'].notna()]

#  Match Ethereum addresses to usernames (or generate placeholder usernames)
address_to_username = df.dropna(subset=['seller.user.username']).drop_duplicates('seller.address')[['seller.address', 'seller.user.username']]
address_to_username_map = dict(zip(address_to_username['seller.address'], address_to_username['seller.user.username']))

def get_or_generate_username(row):
    address = row['seller.address']
    if pd.notna(row['seller.user.username']):
        return row['seller.user.username']
    elif address in address_to_username_map:
        return address_to_username_map[address]
    else:
        return f"user_{str(address)[2:8]}"

df['seller.user.username'] = df.apply(get_or_generate_username, axis=1)

#  Format addresses to make them readable (e.g., Seller_xxxxx, Buyer_xxxxx)
df['seller.address'] = df['seller.address'].astype(str).apply(lambda x: f"Seller_{x[:5]}" if x.lower() != 'nan' else "Unknown")
df['winner_account.address'] = df['winner_account.address'].astype(str).apply(lambda x: f"Buyer_{x[:5]}" if x.lower() != 'nan' else "Unknown")

#  Display missing value counts after cleaning
print(df.isna().sum())

#  Save cleaned dataset
df.to_csv("cleaned_dataset2.csv", index=False)
print("Cleaned dataset successfully saved as 'cleaned_dataset.csv'")


Remaining number of tokens: 59
Filtered dataset shape: (5251176, 15)
sales_datetime             0
id                         0
asset.id                   0
asset.name                 0
asset.collection.name      0
total_price                0
payment_token.name         0
payment_token.usd_price    0
asset.num_sales            0
seller.address             0
seller.user.username       0
winner_account.address     0
Category                   0
usd_price                  0
dtype: int64
Cleaned dataset successfully saved as 'cleaned_dataset.csv'


In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv("opensea_nft_sales.csv", low_memory=False)


df.drop_duplicates(inplace=True)
min_threshold = 50
token_counts = df['payment_token.name'].value_counts(dropna=False)
filtered_tokens = token_counts[token_counts >= min_threshold].index
df = df[df['payment_token.name'].isin(filtered_tokens)]

print(f"Kalan token sayısı: {len(filtered_tokens)}")
print(f"Filtrelenmiş veri boyutu: {df.shape}")

#1
df['asset.name'] = df.apply(
    lambda row: f"Unnamed in {row['asset.collection.name']}" if pd.isna(row['asset.name']) and pd.notna(row['asset.collection.name'])
    else ("Unnamed Asset" if pd.isna(row['asset.name']) else row['asset.name']),
    axis=1
)

#2

df['asset.collection.name'] = df.apply(
    lambda row: f"Collection of {row['asset.name']}" if pd.isna(row['asset.collection.name']) and pd.notna(row['asset.name'])
    else ("Unknown Collection" if pd.isna(row['asset.collection.name']) else row['asset.collection.name']),
    axis=1
)

df.drop(columns=['asset.collection.short_description'], inplace=True)
df.drop(columns=['asset.permalink'], inplace=True)


# 1. Eksik token adlarını en sık geçenle doldur
most_common_token = df['payment_token.name'].mode()[0]
df['payment_token.name'] = df['payment_token.name'].fillna(most_common_token)

# 2. Toplam fiyatı float yap
df['total_price'] = df['total_price'].astype(float)
df['total_price'] = df['total_price'] / 1e18


# 3. Token fiyatlarını oku ve eşleştir
token_price_df = pd.read_csv("token_prices.csv")
token_price_df.columns = token_price_df.columns.str.strip()  # boşlukları temizle
price_dict = dict(zip(token_price_df['token_name'], token_price_df['usd_price']))

# 4. Sadece bilinen token'ları al
df = df[df['payment_token.name'].isin(price_dict.keys())]

# 5. Token başına USD fiyatı doğru olarak ata (çarpma yok!)
df['payment_token.usd_price'] = df['payment_token.name'].map(price_dict)

# 6. USD cinsinden toplam fiyatı hesapla
df['usd_price'] = df['total_price'] * df['payment_token.usd_price']





# Adres - username match
address_to_username = df.dropna(subset=['seller.user.username']).drop_duplicates('seller.address')[['seller.address', 'seller.user.username']]
address_to_username_map = dict(zip(address_to_username['seller.address'], address_to_username['seller.user.username']))

#seller address - username match
def get_or_generate_username(row):
    address = row['seller.address']
    if pd.notna(row['seller.user.username']):
        return row['seller.user.username']
    elif address in address_to_username_map:
        return address_to_username_map[address]
    else:
        return f"user_{str(address)[2:8]}"  # or random/uuid vs.

df['seller.user.username'] = df.apply(get_or_generate_username, axis=1)

# Adresleri stringe çevir
df['seller.address'] = df['seller.address'].astype(str)
df['winner_account.address'] = df['winner_account.address'].astype(str)

# Etiketleri oluştur
df['seller.address'] = df['seller.address'].apply(lambda x: f"Seller_{x[:5]}" if x.lower() != 'nan' else "Unknown")
df['winner_account.address'] = df['winner_account.address'].apply(lambda x: f"Buyer_{x[:5]}" if x.lower() != 'nan' else "Unknown")


print(df.isna().sum())

# Save
df.to_csv("cleaned_dataset.csv", index=False)

Kalan token sayısı: 59
Filtrelenmiş veri boyutu: (5251176, 15)
sales_datetime             0
id                         0
asset.id                   0
asset.name                 0
asset.collection.name      0
total_price                0
payment_token.name         0
payment_token.usd_price    0
asset.num_sales            0
seller.address             0
seller.user.username       0
winner_account.address     0
Category                   0
usd_price                  0
dtype: int64
